In [4]:
import os
import pandas as pd
#load from json .creds/PINECONE_API
import json
with open('.creds/PINECONE_API') as f:
    creds = json.load(f)
    PINECONE_API_KEY = creds['PINECONE_API_KEY']
    PINECONE_ENVIRONMENT = creds['PINECONE_ENVIRONMENT']
    OPENAI_API_KEY = creds['OPENAI_API_KEY']

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [5]:
path_jobs = "data/linkedin/job_posting.json"
path_profiles = "data/linkedin/profiles.json"

import json

# Read profiles.json
with open(path_profiles, 'r') as f:
    profiles = json.load(f)

# Read job_posting.json
with open(path_jobs, 'r') as f:
    job_postings = json.load(f)


In [10]:
type(job_postings)

list

# Create upload function

* We have list with a dictonary as the template.

1. We need to create an embeddigns or summary for each jobposting and candidate
2. Append that embedding to the dictionary (to use the list and create a new key "embedding")


In [11]:
job_postings

[{'id': 1,
  'title': 'Data Analyst',
  'company': 'London Approach',
  'description': "The ideal candidate for the Data Analyst/Financial Analyst position will use their passion for data and analytics to provide insights to the business covering a range of topics. They will be responsible for conducting both recurring and ad hoc analysis for business users.Salary: $100-120kSchedule: 4 days a week onsite & 1 day remote (usually Friday)ResponsibilitiesUnderstand the day-to-day issues that our business faces, which can be better understood with dataCompile and analyze data related to business' issuesDevelop clear visualizations to convey complicated data in a straightforward fashionCollaboratively build financial models and reportsReview and track trendsInvestigate and research discrepanciesMaintain system and code information in systemCompile and analyze dataPrepare and analyze operations and financialsBuild trend reportsQualificationsBachelor's or Master's degree in Accounting or Finan

In [8]:
# Create metadata for profiles
profile_metadata = {id: profile for id, profile in zip(job_postings.keys(), profiles)}

AttributeError: 'list' object has no attribute 'keys'

In [3]:
# Assume that `profile_vectors` and `job_posting_vectors` are dictionaries where
# the keys are the IDs of the profiles/job postings and the values are the corresponding vectors.


# Create metadata for job postings
job_posting_metadata = {id: job_posting for id, job_posting in zip(job_posting_vectors.keys(), job_postings)}

# Upload metadata to Pinecone
pinecone.upsert_metadata(index_name=index_name, metadata=profile_metadata)
pinecone.upsert_metadata(index_name=index_name, metadata=job_posting_metadata)


,title,company,description,onsite_remote,salary,location,criteria,posted_date,link
0,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,Buffalo-Niagara Falls Area,"[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...
1,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"San Jose, CA","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...
2,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Texas, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...
3,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Illinois, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...
4,Entry-Level Data Analyst,The Federal Savings Bank,"The Federal Savings Bank, a national bank and ...",onsite,NaN,"Chicago, IL","[{'Seniority level': 'Entry level'}, {'Employm...",2022-11-17,https://www.linkedin.com/jobs/view/entry-level...
...,...,...,...,...,...,...,...,...,...
2840,Junior Data Analyst,Iris Software Inc.,"Iris's client, one of the world's largest fina...",hybrid,NaN,"Texas, United States","[{'Seniority level': 'Mid-Senior level'}, {'Em...",2022-10-21,https://www.linkedin.com/jobs/view/junior-data...
2841,Data Analyst (SQL),Marwood Group,The Marwood Group (Marwood) is a leading healt...,hybrid,"$75,000.00\r\n -\r\n $95...",New York City Metropolitan Area,"[{'Seniority level': 'Mid-Senior level'}, {'Em...",2022-10-26,https://www.linkedin.com/jobs/view/data-analys...
2842,Data Analyst,SmartSense by Digi,"Join a high-performing, tight-knit team at a f...",hybrid,NaN,"Mishawaka, IN","[{'Seniority level': 'Associate'}, {'Employmen...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...
2843,Data Analyst,Synergy Search,Nashville (Berry Hill) based company looking t...,hybrid,"$85,000.00\r\n -\r\n $95...",Nashville Metropolitan Area,"[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-14,https://www.linkedin.com/jobs/view/data-analys...
